In [1]:
%matplotlib qt

# std
import os
from os import path
import time
import pickle as pkl
from importlib import reload  

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
from src import helpers, configuration, filtering, bandpower, plots, statistics as stats
from src.configuration import cfg, cfg_mr
os.chdir('analysis-adelie')

# mne
import mne

from scipy.io import loadmat  # this is the SciPy module that loads mat-files
from datetime import datetime, date, time

# statistics
from scipy.stats import ttest_ind

# plots
import matplotlib

font = {'family':'DejaVu Sans', 'weight':'normal', 'size':10}
matplotlib.rc('font', **font)

#### Load data

In [172]:
from importlib import reload  
reload(bandpower)
reload(helpers)
reload(configuration)
cfg_mr

{'paths': {'base': 'C:/Users/adeli/OneDrive/Documents/GitHub/data/AlphaTheta',
  'subjects': {'mr': {'prefix': '/MatthieuRicard',
    'recordings': {'baseline': ['baseline1', 'baseline2'],
     'meditation': ['meditation']}}}},
 'columns_to_remove': ['nois', 'sync', 'STI 014'],
 'default_signal_crop': slice(3000, -3000, None),
 'sampling_frequency': 250,
 'bands': {'gamma': [40, 100],
  'beta': [12, 40],
  'alpha': [8, 12],
  'theta': [4, 8],
  'delta': [1, 4]}}

In [3]:
paths = ["../../data/AlphaTheta/MatthieuRicard/MR_20150512_RestingState/MR_12052015.raw",
         "../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Pre_injection_rest/MR_20150511_RestingState.raw", 
         "../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Post_injection_meditation/MR_20150511_Meditation.raw"]

In [7]:
data_baseline1 = mne.io.read_raw_egi(paths[0], preload=False)

Reading EGI header from ../../data/AlphaTheta/MatthieuRicard/MR_20150512_RestingState/MR_12052015.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {nois, sync} ...


In [41]:
data_baseline2 = mne.io.read_raw_egi(paths[1], preload=False)

Reading EGI header from ../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Pre_injection_rest/MR_20150511_RestingState.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {sync} ...


<ipython-input-41-9b3427931c12>:1: RuntimeWarning: Did not find any event code with more than one event.
  data_baseline2 = mne.io.read_raw_egi(paths[1], preload=False)


In [57]:
data_meditation = mne.io.read_raw_egi(paths[2], preload=False)

Reading EGI header from ../../data/AlphaTheta/MatthieuRicard/MR_20150511_Meditation/Post_injection_meditation/MR_20150511_Meditation.raw...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {nois, sync} ...


In [ ]:
sf = 250

#### Preprocess

Data Baseline 1

In [146]:
data_baseline1_df = data_baseline1.to_data_frame(index=['time'], time_format='timedelta')

In [206]:
# Save
data_baseline1_df['22min':'25min'].to_pickle('../../data/AlphaTheta/MatthieuRicard/offline/baseline1.2-raw.pcl')

In [34]:
# length of recording: 48.7458 minutes
len(data_baseline1_df) / 250 / 60

48.7458

In [54]:
epoch_bl1 = data_baseline1_df[:]
data_baseline1_df_epochs = data_baseline1_df.groupby(pd.Grouper(freq='10s'))

In [143]:
test_data = data_baseline1_df['5s':'29s']
test_data['AVG'] = test_data.mean(axis=1)
bp_ricard = bandpower.get_bandpower_epochs_for_all_electrodes(test_data, cfg_mr)
bp_ricard.head()

C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


channel                   E1            E2            E3            E4  \
      time                                                               
gamma 00:00:05  1.544973e-08  1.187223e-08  1.211348e-08  2.695272e-09   
      00:00:15  1.426994e-07  3.699841e-08  2.426037e-08  6.156960e-09   
      00:00:25  8.298446e-08  1.713759e-08  2.029342e-08  1.043966e-08   
beta  00:00:05  3.640034e-02  5.348101e-02  5.944043e-02  1.274169e-02   
      00:00:15  2.921551e-01  1.479748e-01  9.246655e-02  2.286277e-02   

channel                   E5            E6            E7            E8  \
      time                                                               
gamma 00:00:05  5.094975e-09  9.177908e-10  2.801655e-09  1.882564e-10   
      00:00:15  8.504271e-09  2.168009e-09  2.618233e-09  5.078622e-09   
      00:00:25  8.868438e-09  5.540163e-09  2.539302e-09  1.185176e-09   
beta  00:00:05  2.407691e-02  4.709202e-03  1.773545e-02  1.428760e-03   
      00:00:15  3.341841e-02  8.410029e-03  1.502681e-02  2.538604e-02   

channel                   E9           E10  ...          E247          E248  \
      time                                  ...                               
gamma 00:00:05  4.275618e-09  2.311737e-09  ...  6.328789e-08  9.893930e-09   
      00:00:15  1.382652e-08  9.100949e-09  ...  8.017780e-08  9.283261e-08   
      00:00:25  1.338786e-08  5.197419e-09  ...  5.971756e-08  1.157650e-07   
beta  00:00:05  1.216472e-02  1.046020e-02  ...  1.463458e-01  2.454044e-02   
      00:00:15  3.247734e-02  2.968458e-02  ...  2.340126e-01  2.079599e-01   

channel                 E249          E250          E251          E253  \
      time                                                               
gamma 00:00:05  2.702767e-08  4.937642e-08  5.470344e-08  2.490092e-08   
      00:00:15  9.850309e-08  9.318447e-08  9.099729e-08  1.041519e-07   
      00:00:25  7.320837e-08  7.344264e-08  5.979392e-08  9.140924e-08   
beta  00:00:05  9.905987e-02  1.391456e-01  1.633090e-01  1.087655e-01   
      00:00:15  2.570621e-01  2.644736e-01  2.674801e-01  1.958200e-01   

channel                 E254          E255          E256           AVG  
      time                                                              
gamma 00:00:05  2.412339e-08  2.670430e-08  2.020268e-08  9.341263e-09  
      00:00:15  9.908010e-08  7.467092e-08  5.932797e-08  2.279135e-08  
      00:00:25  6.282406e-08  6.406338e-08  4.067162e-08  1.648165e-08  
beta  00:00:05  1.017733e-01  8.962833e-02  9.630654e-02  7.311246e-02  
      00:00:15  2.461553e-01  1.987984e-01  2.094144e-01  7.788213e-02  

[5 rows x 247 columns]

In [144]:
bp_ricard['AVG']

               time    
gamma          00:00:05    9.341263e-09
               00:00:15    2.279135e-08
               00:00:25    1.648165e-08
beta           00:00:05    7.311246e-02
               00:00:15    7.788213e-02
               00:00:25    1.277831e-01
alpha          00:00:05    2.323138e-02
               00:00:15    2.557443e-02
               00:00:25    5.563046e-02
theta          00:00:05    2.709767e-02
               00:00:15    4.718486e-02
               00:00:25    4.214671e-02
delta          00:00:05    2.053230e-01
               00:00:15    9.622944e-02
               00:00:25    9.183151e-02
gamma / beta   00:00:05    1.277657e-07
               00:00:15    2.926389e-07
               00:00:25    1.289814e-07
gamma / alpha  00:00:05    4.020967e-07
               00:00:15    8.911770e-07
               00:00:25    2.962703e-07
gamma / theta  00:00:05    3.447257e-07
               00:00:15    4.830224e-07
               00:00:25    3.910542e-07
gamma / delta  0

In [145]:
dsp = stats.get_dsp_welch(test_data, epoch_size='24s')
dsp['AVG'].plot()

C:\Users\adeli\OneDrive\Documents\GitHub\eeg-meditation\src\statistics.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  axis=1


In [55]:
data_baseline1_df_all = bandpower.get_bandpower_epochs_for_all_electrodes(epoch_bl1, cfg_mr)
data_baseline1_df_all.to_pickle("./data_baseline1_df_all.pkl")

In [106]:
data_baseline1_df_all

channel                           E1            E2            E3  \
              time                                                 
gamma         00:00:00  1.272676e-08  3.139696e-08  2.574487e-08   
              00:00:10  2.424481e-08  1.543658e-08  1.296063e-08   
              00:00:20  1.553556e-07  3.494884e-08  3.069125e-08   
              00:00:30  3.794091e-08  9.634466e-09  1.444760e-08   
              00:00:40  2.795332e-08  5.058870e-09  5.315167e-09   
              00:00:50  3.618453e-08  8.983504e-09  9.265785e-09   
              00:01:00  2.199296e-08  7.300030e-09  9.653570e-09   
              00:01:10  3.917160e-08  3.145410e-08  5.313558e-08   
              00:01:20  2.780964e-08  1.372228e-08  3.846767e-08   
              00:01:30  1.547249e-08  7.725186e-09  1.363568e-08   
              00:01:40  1.529594e-08  5.086620e-09  6.460275e-09   
              00:01:50  1.589311e-08  5.274119e-09  5.710108e-09   
              00:02:00  1.633270e-08  4.185227e-09  2.918579e-09   
              00:02:10  3.140861e-08  1.941955e-08  3.665782e-08   
              00:02:20  2.230239e-08  8.128640e-09  1.044552e-08   
              00:02:30  1.306090e-08  7.411747e-09  5.914807e-09   
              00:02:40  5.456664e-09  3.657466e-09  4.880296e-09   
              00:02:50  2.177340e-08  1.497377e-08  2.239307e-08   
              00:03:00  2.392945e-08  9.379912e-09  7.810976e-09   
              00:03:10  1.110016e-08  4.950682e-09  5.419855e-09   
              00:03:20  2.813813e-08  6.209431e-09  7.107732e-09   
              00:03:30  3.484591e-08  1.020236e-08  5.722006e-09   
              00:03:40  3.050247e-08  1.228501e-08  1.279092e-08   
              00:03:50  1.633816e-08  5.945286e-09  4.215613e-09   
              00:04:00  1.127376e-08  4.219541e-09  4.071518e-09   
              00:04:10  1.421793e-08  3.544518e-09  3.241668e-09   
              00:04:20  1.706054e-08  7.136946e-09  6.177682e-09   
              00:04:30  1.212948e-08  6.499587e-09  4.673590e-09   
              00:04:40  1.990963e-08  5.163287e-09  4.718130e-09   
              00:04:50  4.155287e-08  1.590533e-08  1.136050e-08   
...                              ...           ...           ...   
theta / delta 00:43:50  4.315938e-01  2.177290e-01  1.650464e-01   
              00:44:00  1.401076e+00  3.318715e-01  3.163832e-01   
              00:44:10  5.726265e-01  3.178595e-01  2.576362e-01   
              00:44:20  1.631631e+00  4.158613e-01  2.505565e-01   
              00:44:30  5.502811e-01  3.711631e-01  3.415777e-01   
              00:44:40  7.141080e-01  2.489797e-01  1.822354e-01   
              00:44:50  4.587920e-01  2.011960e-01  1.980074e-01   
              00:45:00  1.220220e+00  2.551180e-01  3.281965e-01   
              00:45:10  7.241200e-01  3.967450e-01  2.991425e-01   
              00:45:20  7.661902e-01  3.012432e-01  2.597508e-01   
              00:45:30  6.212632e-01  2.468487e-01  2.645041e-01   
              00:45:40  7.863329e-01  3.407471e-01  2.711163e-01   
              00:45:50  6.327489e-01  3.178690e-01  2.793481e-01   
              00:46:00  4.537826e-01  2.724036e-01  2.021382e-01   
              00:46:10  7.731409e-01  3.972680e-01  2.416130e-01   
              00:46:20  7.129073e-01  2.650574e-01  2.761607e-01   
              00:46:30  2.249910e-01  1.911836e-01  2.106354e-01   
              00:46:40  6.102737e-01  2.045031e-01  2.741683e-01   
              00:46:50  5.370868e-01  2.612999e-01  1.897688e-01   
              00:47:00  3.617760e-01  1.612096e-01  2.145414e-01   
              00:47:10  8.507761e-01  3.132555e-01  2.443637e-01   
              00:47:20  4.144787e-01  1.826419e-01  1.519948e-01   
              00:47:30  2.768964e-01  9.901742e-02  6.004412e-02   
              00:47:40  2.611307e-01  1.027861e-01  1.030498e-01   
              00:47:50  8.878648e-02  1.724735e-01  2.446155e-01   
              00:48:00  4.969251e-02  4.456077e-02  6.27

Data Baseline 2

In [43]:
data_baseline2_df = data_baseline2.to_data_frame(index=['time'], time_format='timedelta')

In [207]:
# Save
data_baseline2_df['22min':'25min'].to_pickle('../../data/AlphaTheta/MatthieuRicard/offline/baseline2.2-raw.pcl')

MemoryError: 

In [58]:
# length of recording: 64.5172 minutes
len(data_baseline2_df) / 500 / 60

32.2586

In [48]:
epoch_bl2 = data_baseline2_df[:]
data_baseline2_df_epochs = data_baseline2_df.groupby(pd.Grouper(freq='10s'))

In [49]:
data_baseline2_df_all = bandpower.get_bandpower_epochs_for_all_electrodes(epoch_bl2, cfg_mr)
data_baseline2_df_all.to_pickle("./data_baseline2_df_all.pkl")

In [64]:
data_baseline2_df_all

channel                           E1            E2            E3  \
              time                                                 
gamma         00:00:00  9.517354e-09  4.727808e-09  2.641408e-09   
              00:00:10  1.207679e-09  8.217015e-10  4.605547e-10   
              00:00:20  3.887266e-09  1.847910e-09  4.362712e-10   
              00:00:30  7.460844e-10  1.113450e-10  9.182824e-11   
              00:00:40  5.895801e-10  9.584401e-11  6.094261e-11   
              00:00:50  2.003639e-09  9.044227e-10  2.883736e-10   
              00:01:00  2.781725e-09  1.383948e-09  6.153152e-10   
              00:01:10  3.694679e-10  9.713551e-11  7.360942e-11   
              00:01:20  3.818937e-10  6.608718e-11  4.747054e-11   
              00:01:30  1.519095e-09  4.730609e-10  1.802147e-10   
              00:01:40  1.080912e-09  1.996330e-10  1.399703e-10   
              00:01:50  1.351084e-09  2.409243e-10  1.631606e-10   
              00:02:00  6.785747e-10  9.288721e-11  8.948126e-11   
              00:02:10  1.687266e-09  2.064496e-10  1.270099e-10   
              00:02:20  2.895716e-09  3.087307e-10  1.838733e-10   
              00:02:30  3.428836e-09  3.091636e-10  1.733175e-10   
              00:02:40  2.629733e-09  1.654212e-10  1.857394e-10   
              00:02:50  3.514529e-09  4.005414e-10  4.456217e-10   
              00:03:00  2.721565e-09  3.441651e-10  2.977072e-10   
              00:03:10  2.866219e-09  4.073342e-10  4.808596e-10   
              00:03:20  1.749569e-09  3.684445e-10  5.053394e-10   
              00:03:30  2.842560e-09  5.997287e-10  6.316505e-10   
              00:03:40  4.921490e-09  7.387064e-10  5.465223e-10   
              00:03:50  4.081046e-09  6.889767e-10  5.178197e-10   
              00:04:00  8.513628e-09  8.720907e-10  7.571243e-10   
              00:04:10  4.589883e-09  6.637704e-10  5.538132e-10   
              00:04:20  6.613336e-09  9.122764e-10  6.980006e-10   
              00:04:30  6.267636e-09  1.109245e-09  7.413450e-10   
              00:04:40  6.036973e-09  1.166862e-09  1.135866e-09   
              00:04:50  3.675486e-09  7.652795e-10  6.698326e-10   
...                              ...           ...           ...   
theta / delta 00:27:20  5.679421e-01  1.732390e-01  1.216298e-01   
              00:27:30  3.584074e-01  1.176365e-01  9.002533e-02   
              00:27:40  6.115186e-01  1.607669e-01  1.051460e-01   
              00:27:50  2.647284e-01  9.290315e-02  7.524267e-02   
              00:28:00  4.795802e-01  1.403221e-01  1.003657e-01   
              00:28:10  3.594101e-01  1.074551e-01  7.343230e-02   
              00:28:20  1.857827e-01  6.035800e-02  4.626266e-02   
              00:28:30  4.925150e-01  1.490236e-01  1.048413e-01   
              00:28:40  3.996483e-01  1.405292e-01  1.034807e-01   
              00:28:50  3.115276e-01  1.019723e-01  8.112374e-02   
              00:29:00  6.375119e-01  1.960544e-01  1.394894e-01   
              00:29:10  1.050844e+00  5.750023e-01  4.762278e-01   
              00:29:20  1.074108e+00  4.623646e-01  3.116247e-01   
              00:29:30  5.296916e-01  1.654466e-01  1.189703e-01   
              00:29:40  7.174386e-01  2.643403e-01  1.882003e-01   
              00:29:50  6.050277e-01  2.162710e-01  1.343610e-01   
              00:30:00  1.137035e+00  5.320148e-01  3.082458e-01   
              00:30:10  1.095299e+00  5.484449e-01  4.253796e-01   
              00:30:20  1.922399e+00  1.253696e+00  1.150989e+00   
              00:30:30  7.121620e-01  2.520693e-01  1.490602e-01   
              00:30:40  1.456121e+00  7.100666e-01  6.051272e-01   
              00:30:50  1.555665e+00  6.389774e-01  5.103475e-01   
              00:31:00  1.968261e+00  1.505501e+00  1.117934e+00   
              00:31:10  1.328432e+00  7.917725e-01  6.350368e-01   
              00:31:20  1.536871e+00  1.607930e+00  1.767289e+00   
              00:31:30  1.511004e+00  1.238682e+00  1.39

Data Meditation

In [59]:
data_meditation_df = data_meditation.to_data_frame(index=['time'], time_format='timedelta')

In [205]:
# Save
data_meditation_df['22min':'25min'].to_pickle('../../data/AlphaTheta/MatthieuRicard/offline/meditation.2-raw.pcl')

In [173]:
df2 = helpers.load_signal_data('baseline', subject='mr', config=cfg_mr, recording=0)
df2.head()
# bandpower.get_bandpower_epochs_for_all_electrodes(df2, config=cfg_mr)

channel,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,...,E250,E251,E252,E253,E254,E255,E256,nois,sync,STI 014
time,,,,,,,,,,,,,,,,,,,,,
00:05:00,-14.833889,-7.833654,6.050017,11.756153,3.700568,6.234875,-0.447034,13.027895,6.398904,-7.155057,...,-16.506557,-15.424979,1.922269e-09,-12.971774,-14.871799,-19.397732,-3.018945,0.0,0.0,0.0
00:05:00.004000,-13.250150,-8.137880,5.930821,11.407312,2.359433,5.042456,-1.573299,11.774323,5.599617,-7.558903,...,-16.541693,-16.774391,1.922269e-09,-13.483561,-14.738560,-20.086372,-4.638108,0.0,0.0,0.0
00:05:00.008000,-10.604450,-6.838527,5.712236,11.271971,1.998328,4.415260,-2.060224,10.905706,5.296990,-5.266930,...,-14.433163,-15.751824,1.922269e-09,-11.612543,-12.557678,-18.601971,-3.960583,0.0,0.0,0.0
00:05:00.012000,-8.000581,-4.789206,5.284980,11.118541,2.288208,4.295796,-2.041762,10.361368,5.450191,-1.287544,...,-10.869658,-12.793509,1.922269e-09,-8.125639,-9.127651,-15.503771,-1.290252,0.0,0.0,0.0
00:05:00.016000,-6.159351,-3.111765,4.609676,10.703563,2.701478,4.488704,-1.801901,10.062616,5.887663,2.813794,...,-6.689195,-8.669353,1.922269e-09,-4.008587,-5.341031,-11.579882,2.696966,0.0,0.0,0.0


In [65]:
# length of recording:  minutes
len(data_meditation_df) / 250 / 60

53.269866666666665

In [61]:
epoch_meditation = data_meditation_df[:]
data_meditation_df_epochs = data_meditation_df.groupby(pd.Grouper(freq='10s'))

In [62]:
data_meditation_df_all = bandpower.get_bandpower_epochs_for_all_electrodes(epoch_meditation, cfg_mr)
data_meditation_df_all.to_pickle("./data_meditation_df_all.pkl")

In [63]:
data_meditation_df_all

channel                           E1            E2            E3  \
              time                                                 
gamma         00:00:00  6.888879e-06  7.083442e-08  5.675454e-08   
              00:00:10  1.796983e-07  1.127386e-07  9.953267e-08   
              00:00:20  4.220372e-07  1.364264e-07  1.309113e-07   
              00:00:30  2.518425e-07  7.303983e-08  8.356465e-08   
              00:00:40  3.651646e-07  7.453097e-08  6.087076e-08   
              00:00:50  3.244804e-07  1.466026e-07  1.171990e-07   
              00:01:00  2.463555e-07  1.190233e-07  6.968139e-08   
              00:01:10  1.829575e-07  1.948093e-07  1.562262e-07   
              00:01:20  2.080004e-07  1.409195e-07  1.180293e-07   
              00:01:30  2.421261e-07  1.178495e-07  1.309836e-07   
              00:01:40  2.779049e-07  6.351263e-08  1.135824e-07   
              00:01:50  2.195972e-07  1.420002e-07  1.335278e-07   
              00:02:00  2.450669e-07  1.219104e-07  8.393518e-08   
              00:02:10  1.759798e-07  1.388474e-07  1.193046e-07   
              00:02:20  3.215433e-07  8.458818e-08  1.085845e-07   
              00:02:30  2.762041e-07  1.554069e-07  1.778534e-07   
              00:02:40  2.107901e-07  1.493755e-07  1.630330e-07   
              00:02:50  2.487014e-07  1.223489e-07  1.791552e-07   
              00:03:00  2.344296e-07  1.163832e-07  2.125821e-07   
              00:03:10  1.559328e-07  6.400100e-08  6.458252e-08   
              00:03:20  4.539665e-08  1.720063e-08  2.330786e-08   
              00:03:30  2.565809e-07  1.476139e-07  5.604013e-08   
              00:03:40  8.058643e-08  1.019489e-07  4.404105e-08   
              00:03:50  8.947490e-08  7.310123e-08  3.553591e-08   
              00:04:00  5.281899e-08  5.902587e-08  4.063728e-08   
              00:04:10  2.192601e-07  1.280265e-07  1.084483e-07   
              00:04:20  1.542487e-07  1.568657e-07  1.351414e-07   
              00:04:30  2.264539e-07  1.204911e-07  9.766833e-08   
              00:04:40  2.007910e-07  8.198948e-08  6.691766e-08   
              00:04:50  1.617934e-07  7.108782e-08  8.228013e-08   
...                              ...           ...           ...   
theta / delta 00:48:20  2.243215e-01  2.393218e-01  1.767819e-01   
              00:48:30  4.321000e-01  3.058300e-01  2.649513e-01   
              00:48:40  4.124470e-02  2.942318e-02  4.163155e-02   
              00:48:50  4.450378e-02  2.109955e-02  1.784221e-02   
              00:49:00  3.930036e-01  2.416336e-01  3.065765e-01   
              00:49:10  4.067336e-01  3.333535e-01  3.584499e-01   
              00:49:20  8.960212e-01  4.987331e-01  2.762492e-01   
              00:49:30  4.498860e-01  1.611544e-01  1.374056e-01   
              00:49:40  3.671979e-01  2.425371e-01  2.739512e-01   
              00:49:50  7.328168e-01  7.892107e-01  3.801930e-01   
              00:50:00  4.715926e-01  3.977953e-01  3.550615e-01   
              00:50:10  4.274911e-01  4.053981e-01  3.756676e-01   
              00:50:20  4.945237e-01  3.329495e-01  2.862616e-01   
              00:50:30  2.284136e-01  1.724572e-01  1.324344e-01   
              00:50:40  4.978528e-01  2.376956e-01  1.688366e-01   
              00:50:50  1.137437e+00  7.458841e-01  4.632410e-01   
              00:51:00  1.163105e-01  1.456530e-02  2.393116e-02   
              00:51:10  5.939223e-01  2.798177e-01  2.095793e-01   
              00:51:20  7.802230e-01  4.266053e-01  1.746030e-01   
              00:51:30  4.612032e-01  4.293287e-01  4.076130e-01   
              00:51:40  6.671771e-01  3.128138e-01  2.655002e-01   
              00:51:50  3.765904e-01  2.203110e-01  1.967857e-01   
              00:52:00  2.903877e-01  2.364572e-01  1.898899e-01   
              00:52:10  4.262913e-01  2.441478e-01  1.841236e-01   
              00:52:20  3.346091e-01  1.864148e-01  2.127010e-01   
              00:52:30  2.866740e-01  1.974864e-01  1.54

#### Naive metrics

In [67]:
# defining frequency bands
cfg_mr['bands'] = {'gamma': [40, 100], 'beta': [12, 40], 'alpha': [8, 12], 'theta': [4, 8], 'delta': [1, 4]}

# defining ratios
ratioList = ['gamma / beta', 'gamma / alpha', 'gamma / theta', 'gamma / delta', 'beta / alpha', 'beta / theta', 'beta / delta', 'alpha / theta', 'alpha / delta', 'theta / delta']

In [78]:
data_baseline_df_all = pd.concat([data_baseline1_df_all, data_baseline2_df_all])

C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [80]:
# MR's data
merged_bandpower_mr1 = pd.concat([data_baseline1_df_all, data_meditation_df_all], keys=["baseline", "meditation"])
merged_bandpower_mr2 = pd.concat([data_baseline2_df_all, data_meditation_df_all], keys=["baseline", "meditation"])

merged_bandpower_mr = pd.concat([data_baseline_df_all, data_meditation_df_all], keys=["baseline", "meditation"])

C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\adeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To ret

Bandpowers

In [195]:
# bandpowers MR
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr, bands_to_plot=cfg_mr['bands'],
                                        title="Mean spectral band powers with bar errors, baseline vs. meditation for Matthieu Ricard")

In [108]:
# bandpowers MR 1
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr1, bands_to_plot=cfg_mr['bands'],
                                        title="Mean spectral band powers with bar errors, baseline vs. meditation for Matthieu Ricard 1")

In [70]:
# bandpowers MR 2
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr2, bands_to_plot=cfg_mr['bands'],
                                        title="Mean spectral band powers with bar errors, baseline vs. meditation for Matthieu Ricard 2")

Bandpower ratios

In [196]:
# bandpower ratios MR
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr, bands_to_plot=ratioList, 
                                        title="Mean spectral band power ratios with bar errors, baseline vs. meditation for Matthieu Ricard")

In [71]:
# bandpower ratios MR 1
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr1, bands_to_plot=ratioList, 
                                        title="Mean spectral band power ratios with bar errors, baseline vs. meditation for Matthieu Ricard 1")

In [72]:
# bandpower ratios MR 2
plots.plot_bandpower_bar_std_concurrent(merged_bandpower_mr2, bands_to_plot=ratioList, 
                                        title="Mean spectral band power ratios with bar errors, baseline vs. meditation for Matthieu Ricard 2")

Statistical analysis

In [83]:
# MR's ttest
from scipy.stats import ttest_ind
merged_bandpower_mr['AVG'] = merged_bandpower_mr.mean(axis=1)

ttest_mr = merged_bandpower_mr.swaplevel(0,1) \
.groupby(level=0).agg(
    lambda x: ttest_ind(x.xs('baseline', level=1), x.xs('meditation', level=1), equal_var=False, nan_policy='omit')
)

In [84]:
# apply FWER correction
FWER_threshold = 0.05 / 15
p_values_naive_metrics_mr = ttest_mr.applymap(lambda x: x[1])
print('The FWER corrected pvalue is', FWER_threshold)

# download to csv
p_values_naive_metrics_mr[p_values_naive_metrics_mr < FWER_threshold]['AVG']

The FWER corrected pvalue is 0.0033333333333333335


alpha                      NaN
alpha / delta     2.787134e-04
alpha / theta              NaN
beta              1.696984e-80
beta / alpha     2.025029e-114
beta / delta      1.962609e-75
beta / theta     1.078681e-104
delta             1.334041e-27
gamma             4.925673e-48
gamma / alpha              NaN
gamma / beta               NaN
gamma / delta     2.477547e-37
gamma / theta              NaN
theta             2.522105e-25
theta / delta     1.142646e-04
Name: AVG, dtype: float64

In [74]:
# MR's ttest 1
from scipy.stats import ttest_ind
merged_bandpower_mr1['AVG'] = merged_bandpower_mr1.mean(axis=1)

ttest_mr1 = merged_bandpower_mr1.swaplevel(0,1) \
.groupby(level=0).agg(
    lambda x: ttest_ind(x.xs('baseline', level=1), x.xs('meditation', level=1), equal_var=False, nan_policy='omit')
)

In [75]:
# apply FWER correction
FWER_threshold = 0.05 / 15
p_values_naive_metrics_mr1 = ttest_mr1.applymap(lambda x: x[1])
print('The FWER corrected pvalue is', FWER_threshold)

# download to csv
p_values_naive_metrics_mr1[p_values_naive_metrics_mr1 < FWER_threshold]['AVG']

The FWER corrected pvalue is 0.0033333333333333335


alpha            3.241697e-15
alpha / delta             NaN
alpha / theta    2.484410e-15
beta             2.313828e-31
beta / alpha     3.036540e-78
beta / delta     9.822257e-57
beta / theta     1.914531e-65
delta            5.028812e-31
gamma            3.593420e-38
gamma / alpha             NaN
gamma / beta              NaN
gamma / delta    4.092402e-35
gamma / theta             NaN
theta            4.789718e-09
theta / delta    3.131751e-12
Name: AVG, dtype: float64

In [76]:
# MR's ttest 2
from scipy.stats import ttest_ind
merged_bandpower_mr2['AVG'] = merged_bandpower_mr2.mean(axis=1)

ttest_mr2 = merged_bandpower_mr2.swaplevel(0,1) \
.groupby(level=0).agg(
    lambda x: ttest_ind(x.xs('baseline', level=1), x.xs('meditation', level=1), equal_var=False, nan_policy='omit')
)

In [77]:
# apply FWER correction
FWER_threshold = 0.05 / 15
p_values_naive_metrics_mr2 = ttest_mr2.applymap(lambda x: x[1])
print('The FWER corrected pvalue is', FWER_threshold)

# download to csv
p_values_naive_metrics_mr2[p_values_naive_metrics_mr2 < FWER_threshold]['AVG']

The FWER corrected pvalue is 0.0033333333333333335


alpha             9.114866e-73
alpha / delta     6.381051e-41
alpha / theta    2.483595e-106
beta             5.295385e-163
beta / alpha     3.230386e-146
beta / delta      2.304302e-99
beta / theta     6.884648e-147
delta             2.935093e-07
gamma             3.022273e-30
gamma / alpha              NaN
gamma / beta               NaN
gamma / delta     6.622446e-23
gamma / theta              NaN
theta             6.451314e-23
theta / delta     6.071777e-16
Name: AVG, dtype: float64

#### Predefined metrics

In [186]:
egi = pd.read_csv('../../data/EGI-sensorplacement2.xyz', delimiter='\t', usecols=[1,2,3,4], header=None).rename(columns={1: 'X', 2: 'Y', 3:'Z', 4:'electrodes'})
ten_20 = pd.read_csv('../../data/Standard-10-20-Cap81.ced', delimiter='\t', usecols=[1,4,5,6]).rename(columns={'labels': 'electrode'})
ten_20.head()

,electrode,X,Y,Z
0,Fp1,0.950,0.309,-0.0349
1,Fpz,0.999,-0.000,-0.0349
2,Fp2,0.950,-0.309,-0.0349
3,Nz,0.927,-0.000,-0.3750
4,AF9,0.663,0.618,-0.4230


In [187]:
egi.head()

,X,Y,Z,electrodes
0,5.56273,7.05836,-2.13833,E1
1,5.33400,7.94136,0.23710,E2
2,4.25700,8.61151,2.77296,E3
3,2.68245,8.04222,5.31354,E4
4,1.63804,6.33091,7.00984,E5


In [190]:
data_baseline1

AttributeError: 'RawEGI' object has no attribute 'montage'

In [249]:
from src import subject
reload(bandpower)
s = subject.SubjectFactory.get_subject("adelie")
rec1 = s.recordings["baseline"][0]
rec2 = s.recordings["meditation"][1]
rec3 = rec1 + rec2

You are trying to add 2 recordings that don't have the same datatype: baseline!=meditation,


In [251]:
rec1.bandpower_by_epoch()

P3  C3  F3  Fz  F4  C4  P4  Cz  \
gamma         00:00:09.991072 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:09.991082 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:09.991093 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:09.991103 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:09.991113 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:09.991123 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019931 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019948 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019958 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019967 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019977 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019987 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.019997 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.020006 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.020016 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050889 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050906 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050917 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050927 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050947 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050957 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050967 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050976 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.050986 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081392 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081407 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081418 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081429 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081440 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:00:10.081450 00:00:09.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
...                                            ..  ..  ..  ..  ..  ..  ..  ..   
theta / delta 00:01:56.698069 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.698087 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.698098 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.698107 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.698117 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.698126 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728473 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728503 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728524 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728543 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728562 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728589 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728606 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728615 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.728625 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.758048 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.758062 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN   
              00:01:56.758072 00:01:49.991072 NaN NaN NaN NaN NaN NaN NaN NaN 

In [238]:
bandpowers = {}
band_name, band_range = list(cfg['bands'].items())[0]
rec3.raw_signal.groupby(
    [pd.Grouper(level=0), pd.Grouper(freq='10s', level=1)]
).mean()

P3           C3           F3  \
baseline   00:00:09.990297 -1353.244840 -1193.285882 -1036.859285   
           00:00:19.990297 -1381.109065 -1205.668966  -969.686894   
           00:00:29.990297 -1380.357356 -1197.463041  -914.448166   
           00:00:39.990297 -1366.220336 -1174.083459  -854.353488   
           00:00:49.990297 -1327.482183 -1168.278765  -836.500362   
           00:00:59.990297 -1286.432206 -1134.040540  -804.446475   
           00:01:09.990297 -1253.408976 -1112.596073  -754.477162   
           00:01:19.990297 -1229.727877 -1098.582032  -722.718282   
           00:01:29.990297 -1235.714123 -1122.297663  -732.005357   
           00:01:39.990297 -1216.487946 -1097.734059  -687.363220   
           00:01:49.990297 -1197.304675 -1086.571031  -676.930436   
meditation 00:00:09.990297   162.264481   614.975575    97.792569   
           00:00:19.990297   171.835966   604.690725    93.186222   
           00:00:29.990297   128.461397   580.276012    65.017512   
           00:00:39.990297    99.078014   547.509504    88.052634   
           00:00:49.990297    82.987822   500.237253    77.738621   
           00:00:59.990297    70.858977   470.553445    68.943632   
           00:01:09.990297    91.735814   464.123339    96.506417   
           00:01:19.990297    61.009777   427.370790    71.889658   
           00:01:29.990297    61.985216   406.530834    57.707031   
           00:01:39.990297    53.578093   391.388148    33.798975   
           00:01:49.990297    24.110796   390.481639    56.515349   
           00:01:59.990297    21.282530   381.348692    37.583962   
           00:02:09.990297    27.417025   363.350792    39.628147   
           00:02:19.990297    14.563613   347.816810    65.344124   
           00:02:29.990297     2.174506   333.943718    65.723066   
           00:02:39.990297    18.476039   328.237620    61.339448   
           00:02:49.990297     9.245704   324.065153    57.017690   
           00:02:59.990297    14.547065   335.883872    65.596296   
           00:03:09.990297    24.246793   325.647183    83.552450   
...                                 ...          ...          ...   
           00:04:09.990297     6.037173   263.682322    87.408140   
           00:04:19.990297     6.284924   237.014581    67.679042   
           00:04:29.990297     3.761942   222.801594    71.943821   
           00:04:39.990297    10.947169   236.381377    73.517771   
           00:04:49.990297   -13.024147   234.849420   114.809579   
           00:04:59.990297   -42.896022   223.822526   112.931161   
           00:05:09.990297   -44.034876   188.407737    65.477946   
           00:05:19.990297   -24.213548   209.943289    67.753662   
           00:05:29.990297   -31.154908   193.367630    55.152204   
           00:05:39.990297   -39.378803   182.594163    64.250267   
           00:05:49.990297   -59.238292   142.040299    55.229436   
           00:05:59.990297   -49.301371   125.127662    68.152909   
           00:06:09.990297   -38.980027   129.564999    82.657697   
           00:06:19.990297   -51.013974   132.429788    77.226683   
           00:06:29.990297   -44.520473   157.728843    83.023319   
           00:06:39.990297   -28.911739   155.313891    67.225888   
           00:06:49.990297   -46.069223   147.425024    50.438381   
           00:06:59.990297   -74.984984   137.231958    43.391276   
           00:07:09.990297   -94.315905    97.109902    33.040967   
           00:07:19.990297  -110.946296    80.596002    25.325989   
           00:07:29.990297  -107.680889    81.346450    32.899350   
           00:07:39.990297  -124.572521    83.442285    13.306838   
           00:07:49.990297  -312.412830    63.301170   -57.336408   
           00:07:59.990297  -122.861462   200.413180   -47.129202   
           00:08:09.990297   -86.575729   199.803436   -33.067722   
           00:08:19.990297   -52.216712   180.604275     9.770100   
           00:08:29.990297   -15.51